In [1]:
import numpy as np

In [2]:
from keras.preprocessing import sequence

In [3]:
from keras.models import Sequential

In [4]:
from keras.layers import Dense, Dropout, Activation

In [5]:
from keras.layers import Conv1D, GlobalMaxPooling1D

In [ ]:
 !wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2020-12-05 17:55:49--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  23.5MB/s    in 3.6s    

2020-12-05 17:55:53 (22.1 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
!gzip /content/aclImdb_v1.tar.gz

In [6]:
!unzip /content/pos2.zip

Archive:  /content/pos2.zip
  inflating: pos2/0_9.txt            
  inflating: pos2/1_7.txt            
  inflating: pos2/10_9.txt           
  inflating: pos2/100_7.txt          
  inflating: pos2/101_8.txt          
  inflating: pos2/102_10.txt         
  inflating: pos2/103_7.txt          
  inflating: pos2/104_10.txt         
  inflating: pos2/105_7.txt          
  inflating: pos2/106_10.txt         
  inflating: pos2/107_10.txt         
  inflating: pos2/108_10.txt         
  inflating: pos2/109_10.txt         
  inflating: pos2/11_9.txt           
  inflating: pos2/110_10.txt         
  inflating: pos2/111_10.txt         
  inflating: pos2/112_10.txt         
  inflating: pos2/113_10.txt         
  inflating: pos2/114_10.txt         
  inflating: pos2/115_10.txt         
  inflating: pos2/116_10.txt         
  inflating: pos2/117_10.txt         
  inflating: pos2/118_8.txt          
  inflating: pos2/119_10.txt         
  inflating: pos2/12_9.txt           
  inflating: pos2/120_

In [7]:
!unzip /content/neg2.zip

Archive:  /content/neg2.zip
  inflating: neg2/0_3.txt            
  inflating: neg2/1_1.txt            
  inflating: neg2/10_2.txt           
  inflating: neg2/100_3.txt          
  inflating: neg2/101_1.txt          
  inflating: neg2/102_1.txt          
  inflating: neg2/103_1.txt          
  inflating: neg2/104_3.txt          
  inflating: neg2/105_2.txt          
  inflating: neg2/106_2.txt          
  inflating: neg2/107_2.txt          
  inflating: neg2/108_1.txt          
  inflating: neg2/109_2.txt          
  inflating: neg2/11_3.txt           
  inflating: neg2/110_1.txt          
  inflating: neg2/111_4.txt          
  inflating: neg2/112_1.txt          
  inflating: neg2/113_4.txt          
  inflating: neg2/114_4.txt          
  inflating: neg2/115_2.txt          
  inflating: neg2/116_1.txt          
  inflating: neg2/117_3.txt          
  inflating: neg2/118_2.txt          
  inflating: neg2/119_4.txt          
  inflating: neg2/12_1.txt           
  inflating: neg2/120_

In [8]:
import glob

In [9]:
import os

In [10]:
from random import shuffle

In [11]:
def pre_process_data(filepath):
  """
    This is dependent on your training data source but we will try to generalize it as best as possible
  """
  positive_path = os.path.join(filepath, 'pos2')
  negative_path = os.path.join(filepath, 'neg2')

  pos_label = 1
  neg_label = 0

  dataset = []

  for filename in glob.glob(os.path.join(positive_path, '*.txt')):
    with open(filename, 'r') as f:
      dataset.append((pos_label, f.read()))

  for filename in glob.glob(os.path.join(negative_path, '*.txt')):
    with open(filename, 'r') as f:
      dataset.append((neg_label, f.read()))

  shuffle(dataset)

  return dataset


In [12]:
dataset = pre_process_data("/content/")

In [13]:
print(dataset[0])

(1, 'As a psychiatrist specialized in trauma, I find this film a beautiful shown example of a severe psychic trauma, even a trauma. It not only explains the enormous difficulties those people have to cope wither, but that even love is sometimes not enough. But she tries!')


In [ ]:
!pip install nlpia

In [14]:
from nltk.tokenize import TreebankWordTokenizer
from gensim.models.keyedvectors import KeyedVectors



In [ ]:
word_vectors = get_data('w2v', limit=200000)

100%|██████████| 402111/402111 [00:28<00:00, 14009.90it/s]

2020-12-05 18:03:45,357 WARNING:  nlpia.loaders:528:normalize_ext_rename normalize_ext.filepath=/usr/local/lib/python3.6/dist-packages/nlpia/bigdata/googlenews-vectors-negative300.bin.gz


KeyboardInterrupt: ignored

In [15]:
import gensim.downloader as api

In [16]:
word_vecs = api.load("word2vec-google-news-300", return_path=True)

In [17]:
print(word_vecs)

/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [18]:
from gensim.models.keyedvectors import KeyedVectors

In [19]:
word_vectors = KeyedVectors.load_word2vec_format(word_vecs, binary=True, limit=200000)

In [20]:
def tokenize_and_vectorize(dataset):
  tokenizer = TreebankWordTokenizer()
  vectorized_data = []
  expected = []
  for sample in dataset:
    tokens = tokenizer.tokenize(sample[1])
    sample_vecs = []
    for token in tokens:
      try:
        sample_vecs.append(word_vectors[token])

      except KeyError:
        pass # No matching token in the Google w2v vocab
    
    vectorized_data.append(sample_vecs)
  return vectorized_data

In [21]:
def collect_expected(dataset):
  """ Peel of the largest values from the dataset """
  expected = []
  for sample in dataset:
    expected.append(sample[0])
  return expected

In [22]:
vectorized_data = tokenize_and_vectorize(dataset)
expected = collect_expected(dataset)

In [23]:
split_point = int(len(vectorized_data)*.8)

In [24]:
X_train = vectorized_data[:split_point]
y_train = expected[:split_point]
X_test = vectorized_data[split_point:]
y_test = expected[split_point:]

In [25]:
maxlen = 400
batch_size = 32
embedding_dims = 300
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

In [26]:
def pad_trunc(data, maxlen):

  """
    For a given dataset pad with zero vectors or truncate to maxlen
  """

  new_data = []

  # Create a vector of 0s the length of our word vectors
  zero_vector = []
  for _ in range(len(data[0][0])):
    zero_vector.append(0.0)

  for sample in data:
    if len(sample) > maxlen:
      temp = sample[:maxlen]
    elif len(sample) < maxlen:
      temp = sample
      # Append the appropriate number 0 vectors to the list 
      additional_elems = maxlen - len(sample)
      for _ in range(additional_elems):
        temp.append(zero_vector)
    
    else:
      temp = sample
    
    new_data.append(temp)

  return new_data

In [27]:
X_train = pad_trunc(X_train, maxlen)
X_test = pad_trunc(X_test, maxlen)


In [28]:
X_train = np.reshape(X_train, (len(X_train), maxlen, embedding_dims))
y_train = np.array(y_train)


In [29]:
X_test = np.reshape(X_test, (len(X_test), maxlen, embedding_dims))
y_test = np.array(y_test)

In [30]:
print("Build Model ......")
model = Sequential()

# we add a Convolution1D, which will learn filters 
# word group filters of size filter_length:
model.add(Conv1D(filters, kernel_size, padding='valid', activation='relu', strides=1, input_shape=(maxlen, embedding_dims)))

# we use max pooling:
model.add(GlobalMaxPooling1D())

# we add a vanila hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# we project on a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



Build Model ......


In [31]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/2
26/26 [==============================] - 1s 26ms/step - loss: 0.6825 - accuracy: 0.5668 - val_loss: 0.6146 - val_accuracy: 0.6965
Epoch 2/2
26/26 [==============================] - 0s 16ms/step - loss: 0.5286 - accuracy: 0.7478 - val_loss: 0.5320 - val_accuracy: 0.7164


In [33]:
model_structure = model.to_json()
with open("cnn_model.json", "w") as json_file:
  json_file.write(model_structure)

model.save_weights("cnn_weights.h5")
print("Model Saved.")

Model Saved.


In [34]:
from keras.models import model_from_json
with open("cnn_model.json", "r") as json_file:
  json_string = json_file.read()

model = model_from_json(json_string)

model.load_weights("cnn_weights.h5")

In [35]:
sample_1 = "I'm hate that the dismal weather that had me down for so long, when will it break! Ugh, when does happiness return?  The sun is blinding and the puffy clouds are too thin.  I can't wait for the weekend."

In [37]:
# We pass a dummy value in the first element of the tuple just because our helper expects it from the way processed the initial data.  That value won't ever see the network, so it can be whatever.
vec_list = tokenize_and_vectorize([(1, sample_1)])

# Tokenize returns a list of the data (length 1 here)
test_vec_list = pad_trunc(vec_list, maxlen)

test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))
model.predict(test_vec)

array([[0.20449972]], dtype=float32)

In [38]:
model.predict_classes(test_vec)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[0]], dtype=int32)